### 1. From the Users.xml file, find all users which are from Georgia and output to screen their DisplayName only.

In [1]:
from pyspark.sql import SparkSession
import re

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext
user = sc.textFile("Users.xml")

#re.split(r'[ ](?=[A-Z]+\b)', input)

#split_user = user.map(lambda line: re.split(r'Location="*"',line))

GA_users = user.filter(lambda line: re.search(r'Location=".*Georgia | .*GA"',line))

#GA_users = user.filter(lambda line: "Georgia" in line)

DisplayNameFull = GA_users.flatMap(lambda line: re.findall(r'DisplayName=".*?"',line))

displayName = DisplayNameFull.flatMap(lambda line: re.findall(r'".*?"',line))

for i in displayName.collect():
    print(i)




"Nick Larsen"
"gfritz"
"Aleksandr Blekh"
"azoorob"
"ontek"
"Aravind R. Yarram"
"ilya"
"Henry Crutcher"
"Matt Simpson"
"Peter Woolfitt"
"matt biskup"
"Jason W"
"Rafael_Espericueta"
"YC Hu"
"James Wang"
"ryan"
"GGA"
"ode2k"
"Teresa Madsen"
"jpm"
"Mr. Rooter of Southeast GA"
"FranGoitia"
"niru dyogi"
"Mac18"
"Aditya Gogoi"
"Lewis Rodgers"
"cosmosa"
"Jimd"
"DRozen"
"David F"
"David"
"rajb245"
"Sealander"
"Vincent"
"PEBKAC"
"nburn42"
"donlan"
"Ankush Chauhan"
"Scott"
"NAGA"
"rams"
"John Barbour"
"Deepak Shenoy"
"German"
"Franco Piccolo"
"Dean Webb"
"Sam Washburn"
"Myron Slaw"
"rmooney"
"mike_stevs"
"Daniel"
"mLstudent33"
"Lance Ruo Zhang"
"alerera"
"addi wei"
"Rajan"
"Juanjo GA"
"Squ1rr3lz"
"ErinPac"


### 2. Using the Users.xml file, provide the count for all users which joined (CreationDate) in 2017. Output this to the screen.

In [37]:
user_join_2017 = user.filter(lambda line: re.search(r'CreationDate="2017.*"',line))

user_join_2017.count()


14239

### 3. Using the PostHistory file, count the number of Posts that feature the words “Spark” and “Scala”. Output this to the screen. (20 points)

In [2]:
postHistory = sc.textFile("PostHistory.xml")

post = postHistory.filter(lambda line: re.search(r'Text=".*Spark*. | .*Scala*."',line))

post.count()



641

### 4. Using the PostHistory file, provide a total count of the words used by each distinct user. In other words, count all words in all posts for each user and display this to screen. You can only identify users by the UserID.You get 15 bonus points if you get the actual DisplayName of the user.


In [21]:
#Get disPlay name from users.xml
#Since display name follow order, just need to append the list
user = sc.textFile("Users.xml")
postHistory = sc.textFile("PostHistory.xml")
#Get user with Id
displayName_raw = user.flatMap(lambda line: re.findall(r'DisplayName=".*?"',line))
displayName = displayName_raw.flatMap(lambda line: re.findall(r'".*?"',line))


id_raw = user.flatMap(lambda line: re.findall(r'row Id=".*?"',line))
id_nraw = id_raw.flatMap(lambda line: re.findall(r'".*?"',line))

displayName_c = displayName.collect()
id_nraw_c = id_nraw.collect()

user_pair_list = []

for i in range(0,id_raw.count()):
    l = (int(id_nraw_c[i].replace('"', '')),displayName_c[i])
    user_pair_list.append(l)

user_pair = sc.parallelize(user_pair_list)

#filter post with Text tag exist
text_post = postHistory.flatMap(lambda line: re.findall(r'UserId=".*?" Text=".*?"',line))

userId_raw = text_post.flatMap(lambda line: re.findall(r'UserId=".*?"',line))

userId = userId_raw.flatMap(lambda line: re.findall(r'[0-9]+',line))

user_text_raw = text_post.flatMap(lambda line: re.findall(r'Text=".*?"',line))

user_text = user_text_raw.map(lambda line: re.findall(r'".*?"',line))

#pairs = user_text.map(lambda line: (line.split(" ")[0],line))

count = user_text.map(lambda line: (len(line[0].split(" "))))


userId_c = userId.collect()
count_c = count.collect()

pairs_list = []
for i in range(0,userId.count()):
    l = (int(userId_c[i]),count_c[i])
    pairs_list.append(l)

pairs = sc.parallelize(pairs_list)

sum = pairs.reduceByKey(lambda x , y: x + y)

result = user_pair.join(sum)

print("ID\tDisplayName\tNumber of Post")
for i in result.collect():
    print(i)

    

ID	DisplayName	Number of Post
(8, ('"Jon Ericson"', 99))
(24, ('"buruzaemon"', 1294))
(36, ('"statsRus"', 267))
(52, ('"Koba"', 279))
(64, ('"Dan C"', 97))
(84, ('"Rubens"', 17802))
(88, ('"Igor Bobriakov"', 362))
(92, ('"cwharland"', 1718))
(96, ('"ePezhman"', 96))
(104, ('"tanou"', 441))
(108, ('"rapaio"', 24184))
(116, ('"AdrianBR"', 811))
(132, ('"DVK"', 854))
(136, ('"akellyirl"', 260))
(140, ('"Grace Note"', 559))
(156, ('"Marco13"', 3713))
(160, ('"Gregor"', 117))
(168, ('"Mr Lister"', 7))
(172, ('"damienfrancois"', 2212))
(180, ('"Oleksi"', 54))
(192, ('"Cici"', 144))
(200, ('"Tahir Akhtar"', 208))
(208, ('"Joe Fusion"', 223))
(212, ('"Alex Gao"', 71))
(236, ('"organic agave"', 43))
(292, ('"vzn"', 126))
(368, ('"Jeromy Anglim"', 1536))
(384, ('"Atilla Ozgur"', 108))
(404, ('"Ajar"', 18))
(432, ('"Brent Blazek"', 283))
(456, ('"Pensu"', 208))
(464, ('"erogol"', 266))
(480, ('"Ger"', 12))
(484, ('"Neil McGuigan"', 61))
(496, ('"Brown_Dynamite"', 299))
(508, ('"azza-bazoo"', 43))

(26392, ('"Salma"', 184))
(26408, ('"Dawid Cz"', 107))
(26432, ('"JackR"', 51))
(26444, ('"Vivek Khetan"', 1227))
(26448, ('"juan Isaza"', 166))
(26464, ('"TheMethod"', 267))
(26468, ('"tomas"', 23))
(26476, ('"harveyslash"', 608))
(26480, ('"michaelsnowden"', 134))
(26528, ('"Xrave"', 61))
(26596, ('"Society of Data Scientists"', 10369))
(26600, ('"DeepLearningLife"', 71))
(26608, ('"traveh"', 214))
(26612, ('"mallochio"', 818))
(26616, ('"saladi"', 266))
(26676, ('"Bachzen"', 161))
(26692, ('"PhilippPro"', 137))
(26700, ('"c-chavez"', 896))
(26724, ('"foutandabout"', 649))
(26748, ('"Shawn"', 995))
(26752, ('"lU5er"', 286))
(26784, ('"mani"', 52))
(26788, ('"julian"', 2759))
(26800, ('"Rjay155"', 600))
(26828, ('"Benjamin.T"', 138))
(26856, ('"cinqS"', 2037))
(26872, ('"Born2Code"', 497))
(26896, ('"nithin gautham"', 96))
(26956, ('"Chia WJ"', 203))
(26976, ('"Ashish kulkarni"', 268))
(27000, ('"EnDorphin"', 317))
(27108, ('"claudius"', 3067))
(27144, ('"dijksterhuis"', 500))
(27172,

(51188, ('"jeffery.ship"', 100))
(51192, ('"Rohit Jere"', 147))
(51204, ('"Lolade"', 29))
(51208, ('"mouni93"', 4086))
(51216, ('"miguelbadajoz"', 123))
(51244, ('"longbowrocks"', 497))
(51248, ('"Alex"', 186))
(51264, ('"NewbieCoder"', 90))
(51280, ('"Dimitrios Panagopoulos"', 961))
(51308, ('"soeci92"', 302))
(51344, ('"jaiprakash prajapati"', 382))
(51408, ('"Saul Lugo"', 418))
(51416, ('"Thomas Paniagua"', 67))
(51424, ('"Cat C."', 230))
(51440, ('"naser piltan"', 51))
(51472, ('"Lily"', 46))
(51492, ('"GadaaDhaariGeek"', 314))
(51504, ('"J.R."', 309))
(51512, ('"Glenn G."', 390))
(51516, ('"vinay s"', 49))
(51540, ('"Roger T"', 108))
(51604, ('"sssam"', 46))
(51624, ('"Strabonio"', 2156))
(51648, ('"nainometer"', 1286))
(51652, ('"Salsa_virginia"', 89))
(51656, ('"Laura Baker"', 242))
(51668, ('"Tom"', 1977))
(51672, ('"delinco"', 161))
(51696, ('"Ezekiel Kruglick"', 100))
(51708, ('"user2189731"', 275))
(51716, ('"snehal sharma"', 114))
(51724, ('"Nands"', 37))
(51740, ('"Aba "',

(68136, ('"DGS"', 11517))
(68148, ('"Aayush Mahajan"', 352))
(68168, ('"mikesneider"', 129))
(68172, ('"Noah Ziems"', 61))
(68200, ('"ANJALI YADAV"', 817))
(68208, ('"Dhaval Thakkar"', 757))
(68212, ('"Corse"', 334))
(68220, ('"Happy Boy"', 447))
(68232, ('"Fatima"', 440))
(68260, ('"astel"', 321))
(68276, ('"Kang_Kai"', 65))
(68280, ('"Sundara Kesavan"', 91))
(68292, ('"Math graduate student"', 310))
(68304, ('"Hemant"', 85))
(68308, ('"user8314628"', 43))
(68364, ('"cameraguy258"', 99))
(68380, ('"cabe"', 54))
(68392, ('"Olivetree"', 98))
(68396, ('"Guest"', 93))
(68416, ('"Eduardo SantAna da Silva"', 74))
(68440, ('"dadrake"', 558))
(68448, ('"chemssou"', 825))
(68484, ('"Noah"', 23))
(68512, ('"J. Lunger"', 51))
(68532, ('"Kavita Aherwar"', 387))
(68548, ('"nan hu"', 48))
(68556, ('"dfilkovi"', 147))
(68564, ('"A-ar"', 88))
(68572, ('"Srujan K.N."', 305))
(68624, ('"Roulbacha"', 168))
(68664, ('"Savinay_"', 863))
(68692, ('"Andreas"', 334))
(68712, ('"Joe"', 53))
(68728, ('"opsb"',

(12237, ('"William"', 740))
(12241, ('"Alex R."', 925))
(12265, ('"Tim Roberson"', 102))
(12285, ('"Trần Đức Nhuận"', 59))
(12321, ('"harm"', 381))
(12333, ('"Marios Mintzis"', 87))
(12341, ('"davidski"', 1077))
(12345, ('"Arkan"', 2859))
(12377, ('"user2409011"', 984))
(12381, ('"ahjohnston25"', 247))
(12409, ('"Dániel Varga"', 210))
(12417, ('"ctwardy"', 1642))
(12433, ('"Yuval Shachaf"', 142))
(12457, ('"Jane Foster"', 61))
(12469, ('"Renat Gilmanov"', 440))
(12473, ('"jamesmf"', 10428))
(12485, ('"BenoitParis"', 285))
(12497, ('"Justin"', 475))
(12501, ('"user8472"', 127))
(12521, ('"Brendan"', 85))
(12533, ('"keval"', 373))
(12545, ('"KillerSnail"', 35))
(12557, ('"Pranay Mathur"', 804))
(12569, ('"JeeZ"', 102))
(12597, ('"Inês Martins"', 6731))
(12601, ('"Yosi Dahari"', 112))
(12613, ('"vikasreddy"', 143))
(12665, ('"psmith"', 723))
(12717, ('"Andreas"', 85))
(12757, ('"hedgehogues"', 1506))
(12777, ('"dbliss"', 107))
(12785, ('"Rishiraj Surti"', 128))
(12789, ('"Arsen"', 106))
(

...

(57590, ('"Sathiyakugan"', 95))
(57610, ('"Tarlan Ahad"', 1686))
(57614, ('"ch.E"', 1180))
(57618, ('"Roy"', 453))
(57634, ('"Nicolas"', 537))
(57646, ('"Electric"', 54))
(57662, ('"Sadil Khan"', 273))
(57698, ('"Philipps"', 92))
(57706, ('"Shazbots"', 95))
(57722, ('"Aniruddh Khera"', 126))
(57770, ('"David F Rivera Olarte"', 12))
(57786, ('"Naman"', 416))
(57798, ('"Stephen Tetreault"', 254))
(57814, ('"user3508140"', 40))
(57830, ('"Wanting to be anAndroidDevelor"', 168))
(57834, ('"Ashok Suthar"', 799))
(57842, ('"user3500869"', 210))
(57862, ('"user120911"', 103))
(57866, ('"Madhurjya Sharma"', 115))
(57890, ('"faysal"', 205))
(57898, ('"Hexadecimalism"', 2883))
(57906, ('"m3h0w"', 211))
(57914, ('"Tejas Patel"', 90))
(57918, ('"Furqan Hashim"', 108))
(57930, ('"Asghar Chandio"', 36))
(57938, ('"user2552108"', 1124))
(57974, ('"pynewbee"', 448))
(57994, ('"WillZ"', 35))
(57998, ('"Casper"', 406))
(58002, ('"Marcello P."', 411))
(58006, ('"DeepLearner"', 1570))
(58010, ('"M-D Hagel

(76202, ('"vignesh_md"', 178))
(76218, ('"Jimit Jaishwal"', 1093))
(76234, ('"Al Kirk"', 167))
(76242, ('"Gavin Fitzpatrick"', 45))
(76254, ('"Hironsan"', 16))
(76286, ('"Jaahnvi Kulkarni"', 149))
(76298, ('"rspeer"', 114))
(76306, ('"user276806"', 185))
(76338, ('"sailesh"', 82))
(76386, ('"Suresh Pokharel"', 316))
(76394, ('"Aman Agrawal"', 182))
(76422, ('"Guissous Allaeddine"', 9626))
(76426, ('"efirvida"', 607))
(76446, ('"Zion Steiner"', 121))
(76466, ('"Arja"', 172))
(76470, ('"sonnv"', 237))
(76506, ('"TeenPhilosopher"', 188))
(76530, ('"Machupicchu"', 3936))
(76542, ('"Eugene Beresovsky"', 591))
(76546, ('"89f3a1c"', 2822))
(76554, ('"Dan"', 34))
(76574, ('"woody"', 74))
(76578, ('"elescararriba"', 48))
(76590, ('"Anu"', 63))
(76594, ('"Sean"', 931))
(76614, ('"Rinky"', 89))
(76626, ('"Alexey Abramov"', 324))
(76650, ('"Student"', 435))
(76658, ('"fsymao"', 254))
(76674, ('"Rex5"', 2463))
(76686, ('"Amer Farooq"', 77))
(76702, ('"Brad Wheeler"', 88))
(76706, ('"Dibakor Barua"'

(25435, ('"rainer"', 160))
(25439, ('"Newbie"', 51))
(25459, ('"Ted M."', 215))
(25467, ('"user25467"', 201))
(25507, ('"Chris Rackauckas"', 20))
(25515, ('"이해주"', 988))
(25531, ('"suthee"', 49))
(25543, ('"Rapry"', 516))
(25571, ('"setempler"', 34))
(25583, ('"Ulad Kasach"', 160))
(25611, ('"ScottieB"', 1356))
(25623, ('"Jakoer"', 142))
(25631, ('"Avot"', 410))
(25635, ('"Jeremiah"', 42))
(25687, ('"lkavenagh"', 44))
(25691, ('"Back2Basics"', 204))
(25755, ('"franciscojavierarceo"', 10286))
(25759, ('"Grr"', 244))
(25779, ('"Romy"', 72))
(25867, ('"user7117436"', 1282))
(25891, ('"Crashalot"', 119))
(25915, ('"Chris"', 1263))
(25923, ('"Arvind Gurjar"', 60))
(25935, ('"icancto"', 1531))
(25939, ('"Lee Sin"', 138))
(25947, ('"mlibre"', 47))
(25983, ('"user2835098"', 778))
(26019, ('"ebrahimi"', 34903))
(26083, ('"Apurv"', 386))
(26127, ('"Sal"', 1489))
(26163, ('"LUSAQX"', 3591))
(26175, ('"SB2055"', 466))
(26187, ('"lhk"', 417))
(26211, ('"stats0007"', 833))
(26223, ('"michaelosthege"

(54167, ('"LazyCat"', 80))
(54187, ('"Rubio95R"', 736))
(54247, ('"Sahil Thakur"', 73))
(54255, ('"M KS"', 135))
(54271, ('"GRS"', 876))
(54299, ('"Purus"', 144))
(54335, ('"Dave Vennergrund"', 80))
(54343, ('"aerijman"', 238))
(54383, ('"2119"', 167))
(54387, ('"Amir"', 66))
(54395, ('"BrunoGL"', 17452))
(54419, ('"Serg4356"', 99))
(54439, ('"Sebastian Amenabar"', 422))
(54443, ('"LollipopKnight"', 310))
(54471, ('"dsebok"', 89))
(54487, ('"Pulkit Singhal"', 101))
(54491, ('"rohit"', 225))
(54503, ('"T.Tos"', 132))
(54555, ('"김동규"', 47))
(54571, ('"Ari"', 285))
(54575, ('"Adhish Thite"', 160))
(54579, ('"Leo Jiang"', 225))
(54587, ('"Oleksandr Kim"', 4489))
(54607, ('"Peter De Bie"', 24))
(54623, ('"FabioSpaghetti"', 2876))
(54627, ('"NcAdams"', 261))
(54635, ('"user46697"', 96))
(54663, ('"Azeworai"', 276))
(54667, ('"mobupu"', 56))
(54671, ('"Jana Dodson"', 153))
(54683, ('"gs.co"', 273))
(54715, ('"Ali250"', 595))
(54743, ('"sunil4data"', 167))
(54771, ('"subtleseeker"', 898))
(547

(76899, ('"John Doe"', 213))
(76907, ('"Marcelo de Sousa"', 214))
(76915, ('"Deepak Sharma"', 460))
(76923, ('"Shiva"', 213))
(76939, ('"Tom Kealy"', 142))
(76955, ('"thinwybk"', 844))
(77003, ('"Vernica Jain"', 923))
(77015, ('"MarkH"', 531))
(77023, ('"omega"', 514))
(77035, ('"Joao Ferreira"', 956))
(77051, ('"Chicken Man Jones"', 188))
(77067, ('"MLAlex"', 610))
(77075, ('"Prasanth pedda"', 143))
(77107, ('"Daniel Paczuski Bak"', 1291))
(77111, ('"Danny David Leybzon"', 70))
(77143, ('"user10351235"', 90))
(77159, ('"Felipe Araya"', 1294))
(77167, ('"Tsuman"', 115))
(77183, ('"Mach."', 148))
(77227, ('"Benjamin Ricard"', 544))
(77251, ('"2017Algebraist"', 5180))
(77295, ('"Harish Kumar"', 515))
(77311, ('"P K"', 172))
(77319, ('"Pedro Marques"', 93))
(77323, ('"Karampistis Dimitrios"', 439))
(77351, ('"Kuba S"', 34))
(77379, ('"Amanda"', 160))
(77407, ('"goulartnogueira"', 83))
(77415, ('"neeraj04"', 255))
(77455, ('"stormzhou"', 15))
(77471, ('"jaxsonds"', 135))
(77499, ('"feeeper

### 5. GRADUATE STUDENTS: Using the users.xml, comments.xml and PostHistory.xml files, produce a single file that includes the following information: DisplayName, Number of Comments, total Score and Number of posts. This file should have the users (DisplayName) sorted by score, descending from higher to lower. 


In [10]:
user_raw = sc.textFile("Users.xml")
postHistory_raw= sc.textFile("PostHistory.xml")
comment_raw = sc.textFile("Comments.xml")

#Display name
displayName = user_raw.flatMap(lambda line: re.findall(r'DisplayName=".*?"',line))\
                  .flatMap(lambda line: re.findall(r'".*?"',line))

id_nraw = user.flatMap(lambda line: re.findall(r'row Id=".*?"',line))\
         .flatMap(lambda line: re.findall(r'".*?"',line))


displayName_c = displayName.collect()
id_nraw_c = id_nraw.collect()

user_pair_list = []

for i in range(0,id_nraw.count()):
    l = (int(id_nraw_c[i].replace('"', '')),displayName_c[i])
    user_pair_list.append(l)

#key: userID value: displayName
displayName_pair = sc.parallelize(user_pair_list)


#################################
#filter post with Text tag exist
post_raw = postHistory_raw.flatMap(lambda line: re.findall(r'UserId=".*?" Text=".*?"',line))

userId_postHistory = post_raw.flatMap(lambda line: re.findall(r'UserId=".*?"',line))\
                     .flatMap(lambda line: re.findall(r'[0-9]+',line))


user_num_post_raw = userId_postHistory.map(lambda line: (int(line),1))

#key: userID value: number of post
user_num_post_pair = user_num_post_raw.reduceByKey(lambda x, y: x + y )


#############################
#comment files
userId_comment = comment_raw.flatMap(lambda line: re.findall(r'UserId=".*?"',line))\
                  .flatMap(lambda line: re.findall(r'".*?"',line))

#Key: ID value: number of comments
user_numComment = userId_comment.map(lambda line: (int(line.replace('"', '')),1))

num_of_comment = user_numComment.reduceByKey(lambda x, y: x + y)
#get number of comments for each ID



score_comment_pair = comment_raw.flatMap(lambda line: re.findall(r'Score=".*?"',line))\
                  .flatMap(lambda line: re.findall(r'".*?"',line))
userId_comment_collect = userId_comment.collect()
score_comment_collect = score_comment_pair.collect()
score_comment_list = []

for i in range(0,user_numComment.count()):
    s = (int(userId_comment_collect[i].replace('"', '')),int(score_comment_collect[i].replace('"', '')))
    score_comment_list.append(s)
    
#Key: ID value: score of each comment
score_comment_pair = sc.parallelize(score_comment_list)

sum_score = score_comment_pair.reduceByKey(lambda x, y: x + y)

comment_file = num_of_comment.join(sum_score)

result = displayName_pair.join(comment_file).join(user_num_post_pair)

result_sort = result.sortBy(lambda a: -a[1][0][1][1])

test = (["UserID, DisplayName, Score, Numberof comments, Number of posts"])
header = sc.parallelize(test)
header.union(result_sort).coalesce(1, True).saveAsTextFile("Part5")

print("Done")

Done
